# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
dataset_path = "data/mini_sparkify_event_data.json"

In [4]:
df = spark.read.json(dataset_path)
df.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [5]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

First let's define the churn function that will find set 1 when row means churn and 0 otherwise. To find a churn we'll use the `page` when `Downgrade` and `Cancellation Confirmation` event happened.

In [ ]:
def churn(value):
    if value == "Downgrade" or value == "Cancellation Confirmation":
        return 1
    return 0

Now let's apply this function our dataframe.

In [11]:
setChurn = udf(churn, IntegerType())
df = df.withColumn("churn", setChurn(df.page))

So now let's take churned users id's and let's put them to separate collection.

In [25]:
churned_ids = df.filter(df.churn == 1).select("userId").dropDuplicates().collect()

[Row(userId='200002'),
 Row(userId='125'),
 Row(userId='51'),
 Row(userId='124'),
 Row(userId='54'),
 Row(userId='15'),
 Row(userId='155'),
 Row(userId='132'),
 Row(userId='100014'),
 Row(userId='101'),
 Row(userId='11'),
 Row(userId='138'),
 Row(userId='300017'),
 Row(userId='29'),
 Row(userId='69'),
 Row(userId='100021'),
 Row(userId='42'),
 Row(userId='87'),
 Row(userId='73'),
 Row(userId='3'),
 Row(userId='113'),
 Row(userId='30'),
 Row(userId='59'),
 Row(userId='139'),
 Row(userId='28'),
 Row(userId='300018'),
 Row(userId='300012'),
 Row(userId='100022'),
 Row(userId='85'),
 Row(userId='35'),
 Row(userId='16'),
 Row(userId='52'),
 Row(userId='300008'),
 Row(userId='71'),
 Row(userId='98'),
 Row(userId='300013'),
 Row(userId='99'),
 Row(userId='100008'),
 Row(userId='100025'),
 Row(userId='300006'),
 Row(userId='300002'),
 Row(userId='96'),
 Row(userId='300007'),
 Row(userId='100'),
 Row(userId='100006'),
 Row(userId='18'),
 Row(userId='70'),
 Row(userId='100005'),
 Row(userId='61'

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.